## 타이타닉 : 생존률예측

### 1. Data 로드 및 확인

In [4]:
# pandas를 이용해서 csv파일 읽어오기
import pandas as pd
df = pd.read_csv("../datasets/titanic_train.csv")

In [9]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
df.info()
# df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### 2. feature 선택
    - feature 와 label 만으로 이뤄진 데이터프레임

In [ ]:
df.columns.tolist()

In [20]:
# 컬럼 중 Pclass, Sex, Surived, Age만 남김

# drop사용 - (df.columns.tolist()활용)
# df_ti = df.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1)
# df_ti

# loc사용
df_ti = df.loc[: , ["Survived", "Sex", "Age", "Pclass"]]
df_ti

,Survived,Sex,Age,Pclass
0,0,male,22.0,3
1,1,female,38.0,1
2,1,female,26.0,3
3,1,female,35.0,1
4,0,male,35.0,3
...,...,...,...,...
886,0,male,27.0,2
887,1,female,19.0,1
888,0,female,NaN,3
889,1,male,26.0,1


In [23]:
# feature를 X로 저장
# 결과값인 Survived를 y로
X = df_ti.drop(["Survived"], axis=1)
y = df_ti["Survived"]

### 3. 결측치 처리

In [29]:
X.isna().sum()

Sex         0
Age       177
Pclass      0
dtype: int64

In [30]:
# 결측치를 ffill을 이용해서 처리. ffill은 앞의 인덱스 값으로 대체.
X["Age"].fillna(method="ffill", inplace=True)
X.isna().sum()

Sex       0
Age       0
Pclass    0
dtype: int64

### 4. label encoding
    - 문자열자료 존재 확인 후 변환(LabelEncoder | one hot encoding)

In [31]:
# 문자를 숫자로 바꿔준다. LabelEncoder
X.info()
# Sex가 문자열(object)임.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Sex     891 non-null    object 
 1   Age     891 non-null    float64
 2   Pclass  891 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 21.0+ KB


In [34]:
# pd.get_dummies(X["Sex"])
# ㄴ 겟더미스는 열이 많아져서 LabelEncoder 사용
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
labels = encoder.fit_transform(X["Sex"])

X["Sex"] = labels

In [35]:
X

,Sex,Age,Pclass
0,1,22.0,3
1,0,38.0,1
2,0,26.0,3
3,0,35.0,1
4,1,35.0,3
...,...,...,...
886,1,27.0,2
887,0,19.0,1
888,0,19.0,3
889,1,26.0,1


### 5. feature Scaling

In [53]:
# MinMaxScaler를 이용해서 Age를 스케일링

# # case1
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(X)
# df_scaled = scaler.transform(X)
# X_df_scaled = pd.DataFrame(data=df_scaled, columns = X.columns)

# X_df_scaled
# # X_df_scaled.describe()

,Sex,Age,Pclass
0,1.0,0.271174,1.0
1,0.0,0.472229,0.0
2,0.0,0.321438,1.0
3,0.0,0.434531,0.0
4,1.0,0.434531,1.0
...,...,...,...
886,1.0,0.334004,0.5
887,0.0,0.233476,0.0
888,0.0,0.233476,1.0
889,1.0,0.321438,0.0


In [54]:
# case2

from sklearn.preprocessing import MinMaxScaler
import numpy as np


scaler = MinMaxScaler()
age = np.array(X['Age']).reshape(-1, 1)
age = scaler.fit_transform(age)
X['Age'] = age
X

,Sex,Age,Pclass
0,1,0.271174,3
1,0,0.472229,1
2,0,0.321438,3
3,0,0.434531,1
4,1,0.434531,3
...,...,...,...
886,1,0.334004,2
887,0,0.233476,1
888,0,0.233476,3
889,1,0.321438,1


### 5. DecisiontreeClassifier와 cross_val_score를 활용한
### ----학습 및 교차검증----

In [57]:
# cv=5
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

dt_clf = DecisionTreeClassifier(random_state=121)

scores = cross_val_score(dt_clf, X, y, scoring="accuracy", cv= 5)

In [77]:
scores.mean()

0.7889962965287804

---

### 6. 예측 및 평가
### 6.1 test data를 train data와 같이 쭉!
    - 로드
    - feature 선별
    - 결측치처리
    - Label Encoding
    - 정규화(MinMaxScaler)

In [60]:
# test data 로딩
df_test = pd.read_csv("../datasets/titanic_test.csv")
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [63]:
# loc사용하여 features선별
X_test = df_test.loc[: , ["Sex", "Age", "Pclass"]]
X_test.isna().sum()

Sex        0
Age       86
Pclass     0
dtype: int64

In [64]:
# 결측치를 ffill을 이용해서 처리. ffill은 앞의 인덱스 값으로 대체.
X_test["Age"].fillna(method="ffill", inplace=True)
X_test.isna().sum()

Sex       0
Age       0
Pclass    0
dtype: int64

In [66]:
# Label Encoding
encoder = LabelEncoder()
labels = encoder.fit_transform(X_test["Sex"])
X_test["Sex"] = labels
X_test

,Sex,Age,Pclass
0,1,34.5,3
1,0,47.0,3
2,1,62.0,2
3,1,27.0,3
4,0,22.0,3
...,...,...,...
413,1,28.0,3
414,0,39.0,1
415,1,38.5,3
416,1,38.5,3


In [ ]:
scaler = MinMaxScaler()
age = np.array(X['Age']).reshape(-1, 1)
age = scaler.fit_transform(age)
X['Age'] = age
X

In [76]:
# MinMaxScaler를 활용한 Age 정규화
scaler = MinMaxScaler()
age = np.array(X_test["Age"]).reshape(-1,1)
age = scaler.fit_transform(age)
X_test["Age"] = age
X_test

,Sex,Age,Pclass
0,1,0.452723,3
1,0,0.617566,3
2,1,0.815377,2
3,1,0.353818,3
4,0,0.287881,3
...,...,...,...
413,1,0.367005,3
414,0,0.512066,1
415,1,0.505473,3
416,1,0.505473,3


### 6.2 predict 예측! 위에서 이미 모델은 학습함

In [95]:
dt_clf.fit(X, y)
pred_1 = dt_clf.predict(X_test)
len(pred_1)

418

### 6.3 예측 case2 : GirdSearchCV 활용 예측

In [80]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

dt_clf = DecisionTreeClassifier(random_state=121)
parameters = {"max_depth" : [1,2,3], "min_samples_split" : [2,3]}

grid_dtree = GridSearchCV(dt_clf, param_grid=parameters, cv=5, refit=True )
grid_dtree.fit(X, y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=121),
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]})

In [83]:
best_dt_clf = grid_dtree.best_estimator_

In [98]:
pred_2 = best_dt_clf.predict(X_test)
len(pred_2)

418

### 6.4 평가

In [87]:
from sklearn.metrics import accuracy_score
import pandas as pd
y_df = pd.read_csv("../datasets/titanic_gender_submission.csv")
y_test = y_df["Survived"]
y_test

In [99]:
# pred_1
accuracy_score(y_test, pred_1)

0.8301435406698564

In [100]:
# pred_2 : gridsearchcv에서 bestestimation 모델을 통해 예측된 값
accuracy_score(y_test, pred_2)

1.0